In [48]:
import math, time
import numpy as np
import pandas as pd

In [49]:
df = pd.read_csv("taetfp.csv", encoding='big5')

In [50]:
df["代碼"]='00'+df["代碼"].apply(str)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19665 entries, 0 to 19664
Data columns (total 8 columns):
代碼         19665 non-null object
日期         19665 non-null int64
中文簡稱       19665 non-null object
開盤價(元)     19665 non-null float64
最高價(元)     19665 non-null float64
最低價(元)     19665 non-null float64
收盤價(元)     19665 non-null float64
成交張數(張)    19665 non-null object
dtypes: float64(4), int64(1), object(3)
memory usage: 1.2+ MB


In [52]:
df.shape

(19665, 8)

# Using fb prophet

In [53]:
df_submit2 = pd.DataFrame(columns=['ETFid','Mon_ud','Mon_cprice','Tue_ud','Tue_cprice','Wed_ud','Wed_cprice',
                                  'Thu_ud','Thu_cprice','Fri_ud','Fri_cprice'])

In [54]:
df_submit2

,ETFid,Mon_ud,Mon_cprice,Tue_ud,Tue_cprice,Wed_ud,Wed_cprice,Thu_ud,Thu_cprice,Fri_ud,Fri_cprice


In [55]:
symbols = list(set(df["代碼"]))

In [56]:
from fbprophet import Prophet

In [57]:
for i in range(0, 18):
    df4 = pd.DataFrame() 
    symbol2 = symbols[i]
    df3 = df[df["代碼"] == symbols[i]].reset_index()
    df4['ds']=df3["日期"].apply(str).str[0:4]+'-'+df3["日期"].apply(str).str[4:6]+'-'+df3["日期"].apply(str).str[6:8]
    df4['y'] = np.log(df3['收盤價(元)'])            
    m = Prophet(interval_width=0.90) # 調整基本設定值
    m.fit(df4)    
    future = m.make_future_dataframe(periods=5)    
    forecast = m.predict(future)
    forecast[['yhat', 'yhat_lower', 'yhat_upper']]=np.exp(forecast[['yhat', 'yhat_lower', 'yhat_upper']])
    df4['y']=np.exp(df4['y'])
    forecast=pd.concat([forecast, df4['y']], axis=1)
    forecast['yhat2']=forecast['yhat'].shift(1)
    forecast['y2']=forecast['y'].shift(1)
    ud1=1
    ud2=1
    ud3=1
    ud4=1
    ud5=1
    if forecast.iloc[len(forecast)-5]['yhat'] - forecast.iloc[len(forecast)-5]['y2'] <0: ud1= -1
    if forecast.iloc[len(forecast)-4]['yhat'] - forecast.iloc[len(forecast)-4]['yhat2'] <0 : ud2= -1
    if forecast.iloc[len(forecast)-3]['yhat'] - forecast.iloc[len(forecast)-3]['yhat2'] <0 : ud3= -1
    if forecast.iloc[len(forecast)-2]['yhat'] - forecast.iloc[len(forecast)-2]['yhat2'] <0 : ud4= -1
    if forecast.iloc[len(forecast)-1]['yhat'] - forecast.iloc[len(forecast)-1]['yhat2'] <0 : ud5= -1
    
    df_temp2 =  pd.DataFrame([[symbol2,ud1,forecast.iloc[len(forecast)-5]['yhat'],
                                      ud2,forecast.iloc[len(forecast)-4]['yhat'],
                                      ud3,forecast.iloc[len(forecast)-3]['yhat'],
                                      ud4,forecast.iloc[len(forecast)-2]['yhat'],
                                      ud5,forecast.iloc[len(forecast)-1]['yhat']]],
                             columns=['ETFid','Mon_ud','Mon_cprice','Tue_ud','Tue_cprice','Wed_ud','Wed_cprice',
                                  'Thu_ud','Thu_cprice','Fri_ud','Fri_cprice'])
    df_submit2=df_submit2.append(df_temp2)

INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=

In [58]:
df_submit2=df_submit2.sort_values(by=['ETFid'])

In [59]:
df_submit2

,ETFid,Mon_ud,Mon_cprice,Tue_ud,Tue_cprice,Wed_ud,Wed_cprice,Thu_ud,Thu_cprice,Fri_ud,Fri_cprice
0,0050,1,82.928555,-1,82.215487,1,82.941588,1,83.111409,1,83.175894
0,0051,-1,33.910710,-1,33.861209,1,33.912645,1,33.993344,1,34.008801
0,0052,1,53.361472,1,54.040318,-1,53.393211,1,53.550559,1,53.657988
0,0053,1,36.523761,1,36.612833,1,36.642382,1,36.751251,1,36.816092
0,0054,-1,24.372253,-1,24.153284,1,24.510465,1,24.566509,1,24.568764
0,0055,-1,17.331624,-1,17.188153,1,17.353220,1,17.361629,-1,17.351210
0,0056,1,26.330971,-1,26.060480,1,26.416903,1,26.445945,1,26.459584
0,0057,1,51.098842,-1,50.653014,1,51.105985,1,51.260843,1,51.316837
0,0058,-1,47.463478,-1,46.609764,1,47.574531,1,47.728725,1,47.735199
0,0059,-1,42.200220,-1,42.121375,1,42.321633,1,42.405418,-1,42.371317


In [60]:
df_submit2.to_csv('stock_predict2.csv', index=False)